<h3>Importing the libraries</h3>

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pydot

In [ ]:
import os
import matplotlib.pyplot as plt
import pydot
import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.utils.vis_utils import plot_model

<h3>Hyperparameters</h3>

In [ ]:
fc_size = 32
dropout_prob = 0.4
num_iterations = [100, 200, 300]

<h3>Importing the dataset</h3>

In [ ]:
cwd = os.getcwd()
dataset_dir = os.path.join(cwd,'Dataset')
model_dir = os.path.join(cwd,'Model')
graph_dir = os.path.join(cwd,'Graphs')

In [ ]:
train_datagen = ImageDataGenerator(brightness_range=[0.1,0.4],
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   rotation_range = 20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   validation_split = 0.2)
test_datagen = ImageDataGenerator()

In [ ]:
train_dataset = train_datagen.flow_from_directory(os.path.join(dataset_dir,'train'), 
                                                  target_size=(224, 224), 
                                                  batch_size=8,
                                                  color_mode = 'rgb')
test_dataset = test_datagen.flow_from_directory(os.path.join(dataset_dir,'test'), 
                                                target_size=(224, 224),
                                                color_mode = 'rgb')

<h3>Building the model with Softmax</h3>

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (5, 5), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(2,2), strides=2),
    layers.Conv2D(64, (5, 5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=2),
    layers.Flatten(),
    layers.Dense(units=fc_size, activation='relu'),
    layers.Dropout(dropout_prob),
    layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

<h3>Training the model</h3>

In [ ]:
for iterations in num_iterations:
    plot_model(model,to_file=os.path.join(graph_dir,f'Pet_Classification_CNN_{iterations}_Softmax_Adam.png'),show_shapes=True,show_layer_names=True)
    print(f"Training for {iterations} iterations")
    history = model.fit(train_dataset, epochs=iterations, validation_data=test_dataset, verbose=0)
    
    max_loss_train=min(history.history['loss'])*100
    max_acc_val=max(history.history['val_accuracy'])*100
    print(f'Maximum Training Loss is {max_loss_train}')
    print(f'Maximum Validation Accuracy obtained is {max_acc_val}%')
    
    #summarizing history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    title = f'Model Accuracy for {iterations} iterations Softmax Adam'
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'])
    plt.savefig(os.path.join(graph_dir,title+'.png'), bbox_inches='tight')
    plt.show()
    plt.close()
    
    #summarizing history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    title = f'Model Loss for {iterations} iterations Softmax Adam'
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train','Test'])
    plt.savefig(os.path.join(graph_dir,title+'.png'), bbox_inches='tight')
    plt.show()
    plt.close()
    
    model.save(os.path.join(model_dir,f'Pet_Classification_CNN_{iterations}_Softmax_Adam.h5'))